In [1]:
import os
import tensorflow as tf 
from tensorflow.python.client import device_lib

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [19]:
import pandas as pd

resistivity = pd.read_csv("C:\\Users\\swims\\Coding\\.ipynb_checkpoints\\SW캠프_대충_보고서\\Resistivity_data_set.csv")


##### 전처리

In [20]:
print(resistivity.columns)


Index(['Unnamed: 0', 'Number', 'X', 'Y', 'Al', 'Ti', 'Cr', 'Fe', 'Co', 'Ni',
       'Cu', 'Zr', 'Mo', 'W', 'Mn', 'Si', 'Mg', 'Resistance', 'Thickness',
       'Resistivity', 'Ex_resistivity', 'ravg', 'delta', 'dHmix', 'ENavg',
       'dEN', 'N', 'Compo'],
      dtype='object')


In [21]:
resistivity.drop(["Unnamed: 0", "Number","Al","Cr","Fe","Co","Ni","Zr","Mo","W","Mn","Si","N","Compo" ], axis='columns', inplace=True)

In [22]:
X_data = resistivity.iloc[:,[0,1,2,3,4,5,6,9,10,11,12,13]]

In [23]:
X_data.shape

(6548, 12)

In [24]:
Y_data = resistivity.iloc[:,[7,8]]

In [25]:
Y_data

,Resistivity,Ex_resistivity
0,30.553231,23.831108
1,30.681781,24.212788
2,90.658141,70.003844
3,87.521931,67.340687
4,84.786582,65.078390
...,...,...
6543,167.729596,157.509173
6544,169.257832,159.237264
6545,171.227321,161.406609
6546,169.862215,160.241359


In [26]:
X_data

,X,Y,Ti,Cu,Mg,Resistance,Thickness,ravg,delta,dHmix,ENavg,dEN
0,-5,39,12.509605,87.490395,0.000000,0.075,899.28569,0.133004,0.099494,-3.940093,1.864973,0.092632
1,-5,41,11.881834,88.118166,0.000000,0.075,903.06935,0.132753,0.097497,-3.769220,1.866731,0.090601
2,-3,-41,47.061894,52.938106,0.000000,0.242,826.97664,0.146825,0.135981,-8.968923,1.768227,0.139758
3,-3,-39,45.888707,54.111293,0.000000,0.235,822.14956,0.146355,0.136191,-8.939150,1.771512,0.139526
4,-3,-37,44.715521,55.284479,0.000000,0.229,817.32248,0.145886,0.136325,-8.899467,1.774797,0.139216
...,...,...,...,...,...,...,...,...,...,...,...,...
6543,41,3,20.112704,63.936747,15.950549,0.369,1003.42550,0.370837,0.621207,-3.800003,1.749576,0.220601
6544,41,5,19.592147,64.086973,16.320880,0.367,1018.08610,0.376080,0.626673,-3.728841,1.748849,0.222096
6545,41,7,19.071590,64.237200,16.691210,0.366,1032.74660,0.381323,0.631991,-3.659717,1.748121,0.223578
6546,41,9,18.551033,64.387426,17.061541,0.358,1047.40720,0.386566,0.637167,-3.592631,1.747394,0.225048


##### 모델링

In [27]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [29]:
# custom R-squared 함수 정의
def r_squared(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res / (SS_tot + K.epsilon())

# resistivity에서 입력 데이터와 타겟 값을 추출
input_data = resistivity.iloc[:,[0,1,2,3,4,5,6,9,10,11,12,13]].values
target_data = resistivity.iloc[:,[7,8]].values

# 데이터를 훈련 세트, 검증 세트, 테스트 세트로 나누기 (훈련 세트 70%, 검증 세트 15%, 테스트 세트 15%)
train_input, temp_input, train_target, temp_target = train_test_split(input_data, target_data, test_size=0.3, random_state=42)
valid_input, test_input, valid_target, test_target = train_test_split(temp_input, temp_target, test_size=0.5, random_state=42)

# ANN 모델 구성
NN = Sequential()
NN.add(Dense(128, input_dim=12, activation='relu'))
NN.add(Dense(64, activation='relu'))
NN.add(BatchNormalization())
NN.add(Dense(32, activation='relu'))
NN.add(Dense(16, activation='relu'))
NN.add(BatchNormalization())
NN.add(Dense(2, activation='linear'))

NN.summary()

# 모델 컴파일
NN.compile(loss='mean_squared_error', optimizer='adam', metrics=[r_squared])

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# ModelCheckpoint callback to save the best model during training
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, mode='min')

# 모델 학습
history = NN.fit(train_input, train_target, epochs=500, validation_data=(valid_input, valid_target), verbose=1, callbacks=[early_stopping, checkpoint])

# Load the best model saved during training
NN.load_weights('best_model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               1664      
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_2 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 batch_normalization_3 (Batc  (None, 16)               64        
 hNormalization)                                      

In [30]:
test_loss, test_r_squared = NN.evaluate(test_input, test_target, verbose=0)

print("Test Loss(MSE):", test_loss)
print("Test R-squared:", test_r_squared)

Test Loss(MSE): 1.7677961587905884
Test R-squared: 0.9990302920341492


In [31]:
NN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               1664      
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_2 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 batch_normalization_3 (Batc  (None, 16)               64        
 hNormalization)                                      

##### 새로운 값 예측 (기존 데이터 활용)

In [32]:
# 예측할 데이터 준비
new_data = tf.constant(X_data.iloc[-1:])  # 예시 입력 데이터

# 모델을 사용하여 예측 수행
predictions = NN(new_data)

# 예측 결과 출력
print("Model 예측값")
print("Resistivity :",predictions[0][0].numpy(),"\nEx_resistivity :",predictions[0][1].numpy())

Model 예측값
Resistivity : 159.91487 
Ex_resistivity : 149.91791


In [33]:
#정답 값 출력
print("정답 값")
print("Resistivity :",Y_data.iloc[-1,0],"\nEx_resistivity :",Y_data.iloc[-1,1])

정답 값
Resistivity : 165.5041494 
Ex_resistivity : 156.0831481
